Лучше всего из статических моделей сработали geowac_lemmas_none_fasttextskipgram_300_5_2020 и geowac_tokens_none_fasttextskipgram_300_5_2020

Как объединяем?

 ⁃ после кластеризации у нас две таблицы: одна с результатами от geowac_lemmas, другая — от geowac_tokens. Каждую таблицу делим на равные четверти, т.е. по заранее определенным значениям 

 ⁃ В каждой четверти считаем кол-во кластера, записываем словарь. Например, если в рамках одной категории у нас есть кластер 0, в который попало 24 примера и кластер 1 с одним примером, то в словарь просто запишем:
 
 {0: 24, 1: 1}
 
 
 ⁃ Переходим к сравнению работы моделей. Допустим, мы смотрим на то, как распределились примеры эмоционального состояния. Одна модель распределила их по двум кластерам 0 и 1 с распределением 0:24 и 1:1. Другая модель тоже распределила эти примеры по двум кластерам 2 и 3, но в другом соотношении, а именно 2:23 и 3:2. Нам больше нравится первый вариант, поскольку там больше примеров, изначально опреределенных как состояние, которые попали в один кластер. В общем выбираем максимальное совпадение


_______________________________

In [ ]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 78.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=d4e6e72a66f93eb8a4ff68dfbc92ab14c2d2258c7991435eb3471049d460f378
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from collections import Counter
import gensim
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numpy as np
import os
import pandas as pd
from pymorphy2 import MorphAnalyzer
import re
from sklearn.cluster import KMeans
import shutil
import string
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
morph = MorphAnalyzer()

In [ ]:
df = pd.read_excel(r'/content/drive/MyDrive/dataset_probe.xlsx')
v_cols = []
for line in df["контексты"]:
    cl_line = re.sub(r'[^\w\s]','', line.lower())
    words = cl_line.split()
    if "в" in words:
        v_cols.append(f"в {words[words.index('в')+1]}")
df["в + сущ"] = v_cols

In [ ]:
cats_digits = ["a" for i in range(25)] + ["b" for i in range(25)] + ["c" for i in range(25)] + ["d" for i in range(25)]
df["короткое обозначение"] = cats_digits

In [ ]:
df

,контексты,значение,vp,pp,в + сущ,короткое обозначение
0,"В универе очень интересно течет жизнь, постоя...",место,течет в универе,в универе,в универе,a
1,Растить в душе побег уныния -- преступление.,место,растить в душе,в душе,в душе,a
2,В Госдуме тоже пытаются бороться с курением,место,пытаются бороться в Госдуме,в госдуме,в госдуме,a
3,Я старалась подольше оставаться в саду,место,старалась оставаться в саду,в саду,в саду,a
4,"Необходимо добавить, что в Китае банковский п...",место,считается в Китае,в китае,в китае,a
...,...,...,...,...,...,...
95,Во время сильной грозы молния ударила в склад ...,Направление,ударила в склад,в склад оружия,в склад,d
96,"...многие хотят полететь в космос, но потом ос...",Направление,хотят полететь в космос,в космос,в космос,d
97,А на общем снимке его взгляд направлен не в ка...,Направление,направлен не в камеру,в камеру,в камеру,d
98,"Молодая девица смотрела в окно, она ждала весны.",Направление,смотрела в окно,в окно,в окно,d


In [ ]:
def data_for_clustering(some_df, model, lemm_or_form):
    senses_ = []
    cols_ = []
    vecs_ = []
    for sns, wordc in zip(some_df["короткое обозначение"], some_df["в + сущ"]):
        prep, noun = wordc.split()
        if prep in model:
            pvec = model.word_vec(prep)
            if lemm_or_form == "lemmas":
                nvec = model.word_vec(morph.normal_forms(noun)[0])
            elif lemm_or_form == "tokens":
                nvec = model.word_vec(noun)
        wordc_vec_lem = pvec + nvec
        senses_.append(sns)
        cols_.append(wordc)
        vecs_.append(wordc_vec_lem)  
    X_ = np.array(vecs_)
    new_df = pd.DataFrame({"короткое обозначение": senses_,
                   "выражение": cols_,
                   "вектор пред+лемма сущ": vecs_})
    return X_, new_df

In [ ]:
def kmeans_clustering(n, matrix, slcted_columns):
    kmeans = KMeans(n_clusters=n, init = "random", n_init = "auto", max_iter = 500, algorithm='elkan').fit(matrix)
    df_res = slcted_columns.copy()
    kclusters = kmeans.predict(matrix)
    df_res["clusters"] = kclusters
    return df_res, kclusters

In [ ]:
def get_key(d, value):
    for k, v in d.items():
        if v == value:
            return k

Лучше всего сработали модели Geowac

In [ ]:
os.mkdir("geowac_tokens")
shutil.unpack_archive("/content/drive/MyDrive/214.zip", "/content/geowac_tokens")

In [ ]:
tokens_model = gensim.models.KeyedVectors.load('geowac_tokens/model.model')

In [ ]:
X_tokens, new_df_tokens = data_for_clustering(df, tokens_model, "tokens")

<ipython-input-10-01d94511d7f9>:8: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(prep)
<ipython-input-10-01d94511d7f9>:12: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec = model.word_vec(noun)


In [ ]:
results_tokens, clusters_tokens = kmeans_clustering(4, X_tokens, df[["контексты", "короткое обозначение"]])

____________

In [ ]:
os.mkdir("geowac_lemmas")
shutil.unpack_archive("/content/drive/MyDrive/213.zip", "/content/geowac_lemmas")

In [ ]:
lemmas_model = gensim.models.KeyedVectors.load('geowac_lemmas/model.model')

In [ ]:
X_lemmas, new_df_lemmas = data_for_clustering(df, lemmas_model, "lemmas")

<ipython-input-10-01d94511d7f9>:8: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  pvec = model.word_vec(prep)
<ipython-input-10-01d94511d7f9>:10: DeprecationWarning: Call to deprecated `word_vec` (Use get_vector instead).
  nvec = model.word_vec(morph.normal_forms(noun)[0])


In [ ]:
results_lemmas, clusters_lemmas = kmeans_clustering(4, X_lemmas, df[["контексты", "короткое обозначение"]])

_________

Смотрим на последовательность (уникальных) кластеров

Мы сами видим наиболее часто встречающиеся категории в кластерах. Попробуем реализовать это в коде
- проходимся по каждому кластеру, считаем, какие в нем категории и сколько примеров в каждой из них
- новый кластер приобретает имя изначальной категории, которая была наиболее популярной в кластере

In [ ]:
def new_cluster_name(results_df, clusters, X = None):
    new_clusters = {}
    max_cats = []
    results_df["clusters"] = results_df[clusters]
    for cl in results_df.clusters.unique():
        cl_cats = results_df.loc[results_df[clusters] == cl]["короткое обозначение"] #категории, вошедшие в кластер
        max_cat = get_key(Counter(cl_cats), max(Counter(cl_cats).values()))
        new_clusters[cl] = max_cat
    if len(max_cats) == len(set(max_cats)):
        new_clusters_list = []
        for cl in results_df[clusters]:
            if cl in new_clusters.keys():
                new_clusters_list.append(new_clusters[cl])
        results_df["new_clusters"] = new_clusters_list
        return results_df
    else:
        new_cluster_name(results_df, X)

In [ ]:
rl = new_cluster_name(results_lemmas, X_lemmas)

In [ ]:
rt = new_cluster_name(results_tokens, X_tokens)

In [ ]:
def divide_results(n, df_res):
    df_part_shape = df_res.shape[0]//n
    for_slices = [0]
    for i in range(n):
        for_slices.append(df_part_shape * (i + 1))
    
    slices = []
    for lim in for_slices:
        if for_slices.index(lim)+1 <= n:
            slices.append((lim, for_slices[for_slices.index(lim)+1])) 

    df_parts = []
    for slice_ in slices:
        df_parts.append(df_res[slice_[0]:slice_[1]])
    return df_parts

In [ ]:
div_results_tokens = divide_results(4, rt)
div_results_lemmas = divide_results(4, rl)

In [ ]:
best_results = []
for part_tokens, part_lemmas, i in zip(div_results_tokens, div_results_lemmas, range(len(div_results_tokens))):
    cls_counter_tokens = Counter(part_tokens["new_clusters"])
    cls_counter_lemmas = Counter(part_lemmas["new_clusters"])
    max_cl_tokens = max(cls_counter_tokens.values())
    max_cl_lemmas = max(cls_counter_lemmas.values())
    if max_cl_tokens >= max_cl_lemmas:
        best_results.append(part_tokens)
    else:
        best_results.append(part_lemmas)

Ниже видно, что в категориях места, состояния и направления лучшие результаты были получены при кластеризации векторов geowac_tokens, а в категории рода деятельности (профессии) при geowac_lemmas

In [ ]:
best_results[0]

,контексты,короткое обозначение,clusters,new_clusters
0,"В универе очень интересно течет жизнь, постоя...",a,2,a
1,Растить в душе побег уныния -- преступление.,a,2,a
2,В Госдуме тоже пытаются бороться с курением,a,2,a
3,Я старалась подольше оставаться в саду,a,2,a
4,"Необходимо добавить, что в Китае банковский п...",a,2,a
5,"В книге реально описаны события, которые вызы...",a,2,a
6,"В ванной было так жарко, что я сделался красн...",a,3,d
7,С приходом тепла в домах стали появляться ванн...,a,2,a
8,"Кроме того, в фильме не только про этих герое...",a,2,a
9,"Ты когда-нибудь смотрел в цирке не на манеж, а...",a,2,a


In [ ]:
best_results[1]

,контексты,короткое обозначение,clusters,new_clusters
25,Друзья в ужасе замахали руками: это совершенно...,b,0,b
26,"Ксюхины родители в ярости были, но, поскольку ...",b,0,b
27,"Подскочив от неожиданности, директор пришёл в ...",b,0,b
28,"Был в панике, пока не догадался обратиться к к...",b,0,b
29,После стаканчика вина и в депрессии он описал ...,b,0,b
30,"Сказал, что сам оказался в шоке, узнав, что то...",b,0,b
31,В отчаянии сыщик потянул за шнурок звонка.,b,0,b
32,"Неужели ты думала, я буду негодовать на тебя, ...",b,0,b
33,Публика пришла в восторг,b,0,b
34,"Я старалась изменить себя и была в восторге, к...",b,0,b


In [ ]:
best_results[2]

,контексты,короткое обозначение,clusters,new_clusters
50,"А ты, кстати, собираешься серьезно уйти в проф...",c,1,c
51,"Е́сли все в артисты подадутся, кто смотре́ть т...",c,1,c
52,Может мне мою́ мла́дшую тоже в подростки запис...,c,1,c
53,Запи́сываюсь в мамы подростков)),c,1,c
54,Пошла́ девчонка в парикмахеры.,c,1,c
55,"Сле́дующий кандида́т в ректоры, Леони́д Мельни...",c,1,c
56,Евфросинией были пострижены в монахини сестра ...,c,1,c
57,Все три женщины были взяты в заложники вместе ...,c,1,c
58,Если мальчик из хорошей еврейской семьи не игр...,c,1,c
59,"— А кто тебя возьмет в жены, бабуль?",c,1,c


In [ ]:
best_results[3]

,контексты,короткое обозначение,clusters,new_clusters
75,закину сумку и пойду в универ,d,1,c
76,Можно выбросить последнюю пачку сигарет и уеха...,d,1,c
77,Медведи поиграли и вернулись в дом,d,3,d
78,Бросьте кусочек льда в воду,d,3,d
79,Преподаватели часто приходили в дом ректора дл...,d,3,d
80,Голод побуждал многих крестьян отправляться в ...,d,1,c
81,"Не так давно мама призналась, что после основн...",d,3,d
82,"Дима ответил: «Спасибо, милая» и вошел в здани...",d,3,d
83,"Роман заехал за ним, и они двинулись в центр М...",d,3,d
84,Я сунул руку в карман и быстро пересчитал мелочь.,d,3,d


Все работает! Проверим на данных, которые мы анализируем!

In [ ]:
mainresults = pd.read_excel(r'/content/drive/MyDrive/dataset_probe.xlsx', sheet_name = "w2v результаты")

In [ ]:
t_results = mainresults[["значение", "в + сущ", "Geowac_token_model — tokens"]]
l_results = mainresults[["значение", "в + сущ", "Geowac_lemma_model"]]

In [ ]:
cats_digits = ["a" for i in range(25)] + ["b" for i in range(25)] + ["c" for i in range(25)] + ["d" for i in range(25)]
t_results["короткое обозначение"] = cats_digits
l_results["короткое обозначение"] = cats_digits

<ipython-input-56-d9af56c052c7>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t_results["короткое обозначение"] = cats_digits
<ipython-input-56-d9af56c052c7>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_results["короткое обозначение"] = cats_digits


In [ ]:
geo_tokens = new_cluster_name(t_results, "Geowac_token_model — tokens")

<ipython-input-47-70237bb500bc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["clusters"] = results_df[clusters]
<ipython-input-47-70237bb500bc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["new_clusters"] = new_clusters_list


In [ ]:
geo_lemmas = new_cluster_name(l_results, "Geowac_lemma_model")

<ipython-input-47-70237bb500bc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["clusters"] = results_df[clusters]
<ipython-input-47-70237bb500bc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  results_df["new_clusters"] = new_clusters_list


In [ ]:
div_results_tokens = divide_results(4, geo_tokens)
div_results_lemmas = divide_results(4, geo_lemmas)

In [ ]:
best_results = []
for part_tokens, part_lemmas, i in zip(div_results_tokens, div_results_lemmas, range(len(div_results_tokens))):
    cls_counter_tokens = Counter(part_tokens["new_clusters"])
    cls_counter_lemmas = Counter(part_lemmas["new_clusters"])
    max_cl_tokens = max(cls_counter_tokens.values())
    max_cl_lemmas = max(cls_counter_lemmas.values())
    if max_cl_tokens >= max_cl_lemmas:
        best_results.append(part_tokens[["значение", "в + сущ", "new_clusters"]])
    else:
        best_results.append(part_lemmas[["значение", "в + сущ", "new_clusters"]])

In [ ]:
best_results[0]

,значение,в + сущ,new_clusters
0,место,в универе,a
1,место,в душе,a
2,место,в госдуме,a
3,место,в саду,a
4,место,в китае,a
5,место,в книге,a
6,место,в ванной,d
7,место,в домах,a
8,место,в фильме,a
9,место,в цирке,a


In [ ]:
best_results[1]

,значение,в + сущ,new_clusters
25,состояние,в ужасе,b
26,состояние,в ярости,b
27,состояние,в ярость,b
28,состояние,в панике,b
29,состояние,в депрессии,b
30,состояние,в шоке,b
31,состояние,в отчаянии,b
32,состояние,в отчаяние,b
33,состояние,в восторг,b
34,состояние,в восторге,b


In [ ]:
best_results[2]

,значение,в + сущ,new_clusters
50,Профессия,в профессионалы,c
51,Профессия,в артисты,c
52,Профессия,в подростки,c
53,Профессия,в мамы,c
54,Профессия,в парикмахеры,c
55,Профессия,в ректоры,a
56,Профессия,в монахини,c
57,Профессия,в заложники,c
58,Профессия,в учителя,c
59,Профессия,в жены,c


In [ ]:
best_results[3]

,значение,в + сущ,new_clusters
75,Направление,в универ,d
76,Направление,в глушь,d
77,Направление,в дом,d
78,Направление,в воду,d
79,Направление,в дом,d
80,Направление,в города,a
81,Направление,в магазин,d
82,Направление,в здание,d
83,Направление,в центр,d
84,Направление,в карман,d
